<a href="https://colab.research.google.com/github/rmoose/audioML/blob/main/spotify_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Client ID: 7c3728f480b542818dcdb0c3aab098b9
# Client Secret: 1ef29498f2ec49c383f220277efcca8f
# Spotify User ID: 1251592209

In [1]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 5.1 MB/s eta 0:00:00


In [23]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import plotly.express as px


In [15]:
cid='7c3728f480b542818dcdb0c3aab098b9'
secret='1ef29498f2ec49c383f220277efcca8f'
# Spotify User ID: 1251592209


#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [18]:

# Your Spotify API credentials
client_id = cid
client_secret = secret

# Initialize the Spotify API client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Playlist link
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

# List to store all the audio features for each track
all_audio_features = []

for track in sp.playlist_tracks(playlist_URI)["items"]:
    # URI
    track_uri = track["track"]["uri"]

    # Get audio features of the track
    audio_features = sp.audio_features(track_uri)[0]

    # Append the audio features dictionary to the list
    all_audio_features.append(audio_features)

# Create a pandas DataFrame from the list of audio features
df = pd.DataFrame(all_audio_features)

# Save the DataFrame to a CSV file
df.to_csv("audio_features_table.csv", index=False)

# Display the first few rows of the DataFrame
new_df = df.iloc[:, :11]
display(new_df.style.format({'Mean': '{:.2f}'}))


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.868000,0.538000,5,-8.603000,1,0.174000,0.269000,0.000003,0.090100,0.732000,99.968000
1,0.802000,0.832000,11,-4.107000,1,0.043400,0.311000,0.000000,0.081500,0.890000,124.997000
2,0.511000,0.532000,5,-5.745000,1,0.057800,0.177000,0.000000,0.291000,0.350000,138.005000
3,0.552000,0.702000,9,-5.707000,1,0.157000,0.117000,0.000021,0.105000,0.564000,169.994000
4,0.627000,0.879000,9,-3.446000,1,0.095500,0.001930,0.000007,0.064700,0.748000,129.979000
5,0.546000,0.846000,5,-5.719000,1,0.181000,0.013500,0.000000,0.607000,0.740000,162.043000
6,0.841000,0.738000,7,-7.455000,0,0.307000,0.520000,0.000000,0.089200,0.484000,169.918000
7,0.708000,0.737000,1,-4.045000,1,0.043600,0.073900,0.001620,0.095500,0.607000,91.986000
8,0.430000,0.692000,6,-4.234000,1,0.042900,0.254000,0.000004,0.148000,0.401000,156.261000
9,0.671000,0.845000,11,-4.930000,0,0.048000,0.020700,0.000000,0.329000,0.775000,110.056000


In [17]:
 # Axlos Summer Bumps
#playlist_link = "https://open.spotify.com/playlist/5OU4FZ6q3W8TykzidYSzyF"

#top global songs
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"

# Pull out the specific playlist reference
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

# Pull out all the individual track references
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

# List to store all the audio features for each track
all_audio_features = []

for track in sp.playlist_tracks(playlist_URI)["items"]:
    # URI
    track_uri = track["track"]["uri"]

    # Get audio features of the track
    audio_features = sp.audio_features(track_uri)[0]

    # Append the audio features dictionary to the list
    all_audio_features.append(audio_features)

# Create a pandas DataFrame from the list of audio features
df = pd.DataFrame(all_audio_features)



# Save the DataFrame to a CSV file
df.to_csv("audio_features_table.csv", index=False)

# Display the DataFrame
new_df = df.iloc[:, :11]
new_df.head()


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.868,0.538,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.732,99.968
1,0.802,0.832,11,-4.107,1,0.0434,0.31100,0.000000,0.0815,0.890,124.997
2,0.511,0.532,5,-5.745,1,0.0578,0.17700,0.000000,0.2910,0.350,138.005
3,0.552,0.702,9,-5.707,1,0.1570,0.11700,0.000021,0.1050,0.564,169.994
4,0.627,0.879,9,-3.446,1,0.0955,0.00193,0.000007,0.0647,0.748,129.979


In [28]:
# Step 1: Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(new_df)

# Step 2: Perform PCA to reduce dimensions for visualization
pca = PCA(n_components=6)
pca_transformed_df = pca.fit_transform(scaled_data)
# print(pca_transformed_df)

# Get the column names of the transformed data
column_names = pca.get_feature_names()
print(column_names)

# pca_transformed_df.columns['PC1','PC2','PC3','PC4','PC5','PC6']

# Plot the scree plot
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)
print(cumulative_explained_variance)

plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, label='Explained Variance Ratio')
plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', color='r', label='Cumulative Explained Variance')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance')
plt.title('Scree Plot')
plt.legend()
plt.xticks(range(1, len(explained_variance_ratio) + 1))
plt.show()


fig = px.scatter(pca_transformed_df, x=0, y=1, title='PCA plot')
fig.show()


AttributeError: ignored